#Data Cleaning

In [5]:
import sys
sys.path.append('/Users/alexsutherland/Dropbox/Insight/WeightLossReddit/')
import os
os.chdir('/Users/alexsutherland/Dropbox/Insight/WeightLossReddit/')
import datetime
import unicodedata
import redditDataIO
import pandas as pd

First, let's load the data:

In [2]:
fullData = redditDataIO.loadData()
fullData.describe()

OperationalError: (OperationalError) (2003, "Can't connect to MySQL server on 'localhost' ([Errno 61] Connection refused)") None None

In [3]:
def convertWeightToPounds(data):
    print 'Distribution of weight units before converting weight to pounds:'
    print data['weightUnit'].value_counts()
    
    kgIndexes = data.weightUnit == 'kg'
    data.ix[kgIndexes,['startWeight','endWeight','weightChange']] = data.ix[kgIndexes,['startWeight','endWeight','weightChange']].applymap(lambda x: float(x)*2.2)
    data.ix[kgIndexes,'weightUnit'] = 'lb'
    
    poIndexes = data.weightUnit == 'po'
    data.ix[poIndexes,'weightUnit'] = 'lb'
    
    print 'Distribution of weight units before converting weight to pounds:'
    print data['weightUnit'].value_counts()
    
    return data

Converting all of the kg weight units to lbs, using a multiplication factor of 2.2 lbs/kg

In [4]:
cleanedData = convertWeightToPounds(fullData)

Distribution of weight units before converting weight to pounds:
lb    3795
      1243
kg     122
po     117
dtype: int64
Distribution of weight units before converting weight to pounds:
lb    4034
      1243
dtype: int64


All of our weight units are in lbs.

If the row does not have a weight unit, we can try to figure out its unit and fill in an appropriate unit:

In [5]:
def fillInMissingWeightUnit(data):
    print 'Distribution of weight units before filling in missing weight units:'
    print data['weightUnit'].value_counts()
    
    #Full in ones that mention lbs
    dataWithMissingWeightUnit = data.ix[data['weightUnit'] == '',]
    
    initialRowsWithoutWeightUnit = dataWithMissingWeightUnit.shape[0]
    
    foundLbsArray = dataWithMissingWeightUnit.ix[dataWithMissingWeightUnit['title'].map(lambda x: True if x.lower().find('lb') > 0 else False),'title']
    
    data.ix[foundLbsArray.index,'weightUnit'] = 'lb'
    
    #Fill in data where the weight could not be kgs
    dataWithMissingWeightUnit = data.ix[data['weightUnit'] == '',]
    
    largeWeightArray = dataWithMissingWeightUnit.ix[dataWithMissingWeightUnit['startWeight'].map(lambda x: True if x > 140 else False),'title']
    
    data.ix[largeWeightArray.index,'weightUnit'] = 'lb'
    
    finalRowsWithoutWeightUnit = data.ix[data['weightUnit'] == '',].shape[0]
    
    print 'Filled in ' + str(initialRowsWithoutWeightUnit - finalRowsWithoutWeightUnit) + ' samples with new WeightUnits. Shape: ' + str(data.shape)
    print ''
    print 'Distribution of weight units after filling in missing weight units:'
    print data['weightUnit'].value_counts()
    return data

In [6]:
cleanedData = fillInMissingWeightUnit(cleanedData)

Distribution of weight units before filling in missing weight units:
lb    4034
      1243
dtype: int64
Filled in 1159 samples with new WeightUnits. Shape: (5277, 22)

Distribution of weight units after filling in missing weight units:
lb    5193
        84
dtype: int64


At this point, we've converted the weight units and filled in missing units. If the units are still not there, we should remove the row:

In [7]:
def removeMissingWeightRows(data):
    initialRows = data.shape[0]    

    dropIndexes = data['weightUnit'] == ''
    dropIndexes = dropIndexes[dropIndexes == True].index.map(lambda x: int(x))

    data = data.drop(dropIndexes, axis=0)
    
    print 'Removed ' + str(initialRows - data.shape[0]) + ' samples without WeightUnits. Shape: ' + str(data.shape)

    return data

In [8]:
cleanedData = removeMissingWeightRows(cleanedData)

Removed 84 samples without WeightUnits. Shape: (5193, 22)


Now, we turn our attention to converting the time elapsed to seconds (or Epoch time):

In [9]:
def convertTimeElapsedToEpoch(data):

    conversionFactors = {'mo': 2600000, 'da': 86400,'ye': 31504000, 'yr': 31504000, 'we': 604800}
    
    data['timeElapsedEpoch'] = data.apply(lambda x: x['timeElapsed'] * conversionFactors[(x['timeUnit'].lower()[0:2])] , axis=1)
    
    print 'Added column for time elapsed in Epoch'
    
    return data

Running this conversion:

In [10]:
cleanedData = convertTimeElapsedToEpoch(cleanedData)

Added column for time elapsed in Epoch


Let's clean up the titles by converting everything to ascii

In [11]:
def cleanTitles(data):
    
    data['title'] = data['title'].map(lambda x: unicodedata.normalize('NFKD', unicode(x, 'utf-8')).encode('ascii','ignore'))
    data['userText'] = data['userText'].map(lambda x: unicodedata.normalize('NFKD', unicode(x, 'utf-8')).encode('ascii','ignore'))
    
    return data

In [12]:
cleanedData = cleanTitles(cleanedData)
cleanedData.head(2)

,index,permalink,userText,name,author,url,num_comments,endWeight,title,created_utc,...,weightChange,over_18,timeElapsed,gender,timeUnit,startWeight,age,currentBMI,previousBMI,timeElapsedEpoch
0,0,/r/progresspics/comments/q39ce/2_month_check_i...,check in 2011/12/9 272lbs (123.4kg) &gt; 2012...,t3_q39ce,psywiped,http://imgur.com/a/TYPbt,8,8.8,2 Month check in 2011/12/9 272lbs (123.4kg) &g...,1330040000,...,-261.8,0,2,M,Month,270.6,28,0.622837,19.1522,5200000
1,1,/r/progresspics/comments/rh4q6/302_275_in_a_mo...,"and a half. M/24/5'11"". Only 85 to go! (Warni...",t3_rh4q6,audiophile8706,http://imgur.com/a/SK54I,4,275.0,302 -&gt; 275 in a month and a half. M/24/5'11...,1332910000,...,-27.0,0,1,M,month,302.0,24,39.277900,43.1343,2600000


Converting the genders to the same format: Male = 'M' and Female = 'F':

In [13]:
def convertMaleFemaleToMF(data):
    print 'Converting coding to Male-->M and Female-->F'
    data['gender'] = data['gender'].map(lambda x: x[0].upper())
    
    return data

In [14]:
cleanedData = convertMaleFemaleToMF(cleanedData)

Converting coding to Male-->M and Female-->F


Let's get the time units into the same formats (ex: Year should equal ye)

In [15]:
def standardizeTimeUnits(dataTemp):
    data = dataTemp.copy()
    print 'Distribution of time units before standardizing:'
    print data['timeUnit'].value_counts()
    print ''
    
    data['timeUnit'] = data['timeUnit'].map(lambda x: x[0:2].upper())
    data.ix[data['timeUnit'] == 'YR','timeUnit'] = 'YE'
    
    print 'Distribution of time units after standardizing:'
    print data['timeUnit'].value_counts()
    
    return data

In [16]:
cleanedData = standardizeTimeUnits(cleanedData)

Distribution of time units before standardizing:
month    3197
years     691
year      575
Month     320
weeks     188
yr         57
Year       53
Years      38
week       29
Weeks      24
MONTH      10
Week        3
YEAR        2
YEARS       2
Yr          2
YR          2
dtype: int64

Distribution of time units after standardizing:
MO    3527
YE    1422
WE     244
dtype: int64


Removing any row that doesn't have a time unit

In [17]:
def removeDataWithoutTimeUnit(data):
    print 'Distribution of time units before dropping empty time units:'
    print data['timeUnit'].value_counts()
    
    initialRows = data.shape[0]    
    
    dropIndexes = data['timeUnit'].map(lambda x: x is None)
    dropIndexes = dropIndexes[dropIndexes == True].index.map(lambda x: int(x))

    data = data.drop(dropIndexes, axis=0)
    
    print 'Removed ' + str(initialRows - data.shape[0]) + ' samples without TimeUnits. Shape: ' + str(data.shape)
    print ''
    print 'Distribution of time units before dropping empty time units:'
    print data['timeUnit'].value_counts()
    return data

In [18]:
cleanedData = removeDataWithoutTimeUnit(cleanedData)

Distribution of time units before dropping empty time units:
MO    3527
YE    1422
WE     244
dtype: int64
Removed 0 samples without TimeUnits. Shape: (5193, 23)

Distribution of time units before dropping empty time units:
MO    3527
YE    1422
WE     244
dtype: int64


Finally, we output the data to a SQL table called "cleanedWeightLossData"

In [19]:
import redditDataIO

redditDataIO.exportDataToSQL(cleanedData, 'cleanedWeightLossData')

Index([u'index', u'permalink', u'userText', u'name', u'author', u'url', u'num_comments', u'endWeight', u'title', u'created_utc', u'height', u'weightUnit', u'score', u'weightChange', u'over_18', u'timeElapsed', u'gender', u'timeUnit', u'startWeight', u'age', u'currentBMI', u'previousBMI', u'timeElapsedEpoch'], dtype='object')
Exported Data to SQL Table (Name: cleanedWeightLossData)


/Users/alexsutherland/anaconda/lib/python2.7/site-packages/pymysql/cursors.py:134: Warning: Unknown table 'redditweightdatabase.cleanedweightlossdata'
  result = self._query(query)
